## Classification
Put simply, classification is the task of predicting a label for a given observation. For example: you are given certain physical descriptions of an animal, and your taks is to classify them as either a dog or a cat. Here, we will classify iris flowers.

As we will see later, we will use different classifiers and at the end of this notebook, we will compare them. We will define our accuracy function right now to get it out of the way. We will use a simple accuracy function that returns the ratio of the number of correctly classified observations to the total number of predictions.

In [1]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [4]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

Get the data first

In [5]:
iris = dataset("datasets", "iris")

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [6]:
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]

150-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [7]:
X

150×4 Matrix{Float64}:
 5.1  3.5  1.4  0.2
 4.9  3.0  1.4  0.2
 4.7  3.2  1.3  0.2
 4.6  3.1  1.5  0.2
 5.0  3.6  1.4  0.2
 5.4  3.9  1.7  0.4
 4.6  3.4  1.4  0.3
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.9  3.1  1.5  0.1
 5.4  3.7  1.5  0.2
 4.8  3.4  1.6  0.2
 4.8  3.0  1.4  0.1
 ⋮              
 6.0  3.0  4.8  1.8
 6.9  3.1  5.4  2.1
 6.7  3.1  5.6  2.4
 6.9  3.1  5.1  2.3
 5.8  2.7  5.1  1.9
 6.8  3.2  5.9  2.3
 6.7  3.3  5.7  2.5
 6.7  3.0  5.2  2.3
 6.3  2.5  5.0  1.9
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3
 5.9  3.0  5.1  1.8

In [11]:
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)

150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In classification, we often want to use some of the data to fit a model, and the rest of the data to validate (commonly known as `training` and `testing` data). We will get this data ready now so that we can easily use it in the rest of this notebook.

In [21]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [22]:
?randsubseq

search: randsubseq randsubseq! RandomSub StratifiedRandomSub



```
randsubseq([rng=GLOBAL_RNG,] A, p) -> Vector
```

Return a vector consisting of a random subsequence of the given array `A`, where each element of `A` is included (in order) with independent probability `p`. (Complexity is linear in `p*length(A)`, so this function is efficient even if `p` is small and `A` is large.) Technically, this process is known as "Bernoulli sampling" of `A`.

# Examples

```jldoctest
julia> rng = MersenneTwister(1234);

julia> randsubseq(rng, 1:8, 0.3)
2-element Vector{Int64}:
 7
 8
```


In [23]:
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

49-element Vector{Int64}:
   1
   2
  11
  13
  14
  16
  17
  19
  22
  23
  27
  29
  31
   ⋮
 118
 120
 126
 129
 131
 134
 135
 136
 140
 141
 143
 144

We will need one more function, and that is the function that will assign classes based on the predicted values when the predicted values are continuous.

In [24]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

### 🟣 Method 1: Lasso

In [25]:
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])

Least Squares GLMNet Cross Validation
71 models for 4 predictors in 10 folds
Best λ 0.013 (mean loss 0.050, std 0.008)

In [26]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])
mylambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda]);

In [27]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)

49×1 Matrix{Float64}:
 0.9155468195558165
 0.9356486341723336
 0.9039536758116612
 0.8743803717238496
 0.8689561737232818
 1.0164392934003705
 1.0094728589639057
 0.9822392800048685
 1.076744737249922
 0.8915630433746283
 1.1053552227388006
 0.9054959122475579
 0.9828186978418328
 ⋮
 3.0642642682488184
 2.4203826751214716
 2.6993250784987994
 2.919451533512614
 2.769102011819645
 2.408789531377316
 2.450370067140192
 3.024225968921839
 2.832077840610081
 3.103256320858066
 2.7443207469455793
 3.0775649783349257

In [28]:
predictions_lasso = assign_class.(predictions_lasso)
findaccuracy(predictions_lasso,y[testids])

0.9183673469387755

### 🟣 Method 2: Ridge
We will use the same function but set alpha to zero.

In [29]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

0.9387755102040817

### 🟣 Method 3: Elastic Net
We will use the same function but set alpha to 0.5 (it's the combination of lasso and ridge).

In [30]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

0.9183673469387755

### 🟣 Method 4: Decision Trees
We will use the package `DecisionTree`

In [31]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [32]:
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

0.9183673469387755

### 🟣 Method 5: Random Forests
The `RandomForestClassifier` is available through the `DecisionTree` package as well.

In [33]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 5.25
Avg Depth:  3.7

In [34]:
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

0.9183673469387755

### 🟣 Method 6: Using a Nearest Neighbor method
We will use the `NearestNeighbors` package here.

In [35]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

KDTree{StaticArrays.SVector{4, Float64}, Euclidean, Float64}
  Number of points: 101
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true

In [36]:
queries = X[testids,:]

49×4 Matrix{Float64}:
 5.1  3.5  1.4  0.2
 4.9  3.0  1.4  0.2
 5.4  3.7  1.5  0.2
 4.8  3.0  1.4  0.1
 4.3  3.0  1.1  0.1
 5.7  4.4  1.5  0.4
 5.4  3.9  1.3  0.4
 5.7  3.8  1.7  0.3
 5.1  3.7  1.5  0.4
 4.6  3.6  1.0  0.2
 5.0  3.4  1.6  0.4
 5.2  3.4  1.4  0.2
 4.8  3.1  1.6  0.2
 ⋮              
 7.7  3.8  6.7  2.2
 6.0  2.2  5.0  1.5
 7.2  3.2  6.0  1.8
 6.4  2.8  5.6  2.1
 7.4  2.8  6.1  1.9
 6.3  2.8  5.1  1.5
 6.1  2.6  5.6  1.4
 7.7  3.0  6.1  2.3
 6.9  3.1  5.4  2.1
 6.7  3.1  5.6  2.4
 5.8  2.7  5.1  1.9
 6.8  3.2  5.9  2.3

In [37]:
idxs, dists = knn(kdtree, queries', 5, true)

([[11, 3, 25, 17, 6], [22, 30, 8, 16, 1], [32, 17, 12, 31, 4], [8, 30, 22, 1, 2], [24, 27, 7, 1, 2], [21, 10, 4, 20, 32], [32, 21, 12, 4, 20], [4, 32, 13, 19, 21], [12, 31, 11, 3, 32], [5, 1, 23, 27, 3]  …  [89, 67, 72, 82, 86], [91, 69, 68, 76, 80], [72, 67, 89, 70, 84], [56, 48, 85, 87, 88], [68, 56, 76, 80, 93], [70, 67, 72, 84, 74], [77, 97, 95, 82, 86], [96, 82, 77, 69, 86], [83, 101, 56, 88, 94], [82, 86, 96, 69, 67]], [[0.09999999999999998, 0.1414213562373093, 0.14142135623730964, 0.14142135623730995, 0.17320508075688762], [0.14142135623730964, 0.14142135623730986, 0.17320508075688784, 0.22360679774997896, 0.30000000000000016], [0.10000000000000053, 0.2828427124746193, 0.3316624790355405, 0.3316624790355405, 0.3464101615137753], [0.17320508075688815, 0.19999999999999998, 0.20000000000000037, 0.26457513110645897, 0.26457513110645925], [0.244948974278318, 0.31622776601683816, 0.34641016151377546, 0.5000000000000003, 0.519615242270663], [0.3605551275463992, 0.5477225575051663, 0.61

In [38]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(string(argmax(possible_labels[i])))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

0.9591836734693877

### 🟣 Method 7: Support Vector Machines
We will use the `LIBSVM` package here.

In [39]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

101-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [40]:
model = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Vector{Int64}, Matrix{Float64}}(36, Int32[4, 16, 16], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [5.8 5.5 … 6.5 5.9; 4.0 4.2 … 3.0 3.0; 1.2 1.4 … 5.2 5.1; 0.2 0.2 … 2.0 1.8], Int32[10, 21, 26, 29, 33, 35, 37, 38, 40, 42  …  87, 88, 89, 90, 92, 94, 95, 98, 99, 101], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 5.5), LIBSVM.SVMNode(1, 4.5), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 7.0), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 5.0), LIBSVM.SVMNode(1, 6.1)  …  LIBSVM.SVMNode(1, 6.2), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 7.2), LIBSVM.SVMNode(1, 7.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.0), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 5.9)]), 0.0, [0.2502430089393789 0.5923362118086088; 0.0 0.13977782155760943; … ; -0.0

In [41]:
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

0.9387755102040817

Putting all the results together:

In [42]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso,ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge,ytest)
overall_accuracies[3] = findaccuracy(predictions_EN,ytest)
overall_accuracies[4] = findaccuracy(predictions_DT,ytest)
overall_accuracies[5] = findaccuracy(predictions_RF,ytest)
overall_accuracies[6] = findaccuracy(predictions_NN,ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM,ytest)
hcat(methods, overall_accuracies)

7×2 Matrix{Any}:
 "lasso"  0.918367
 "ridge"  0.938776
 "EN"     0.918367
 "DT"     0.918367
 "RF"     0.918367
 "kNN"    0.959184
 "SVM"    0.938776

# Finally...
After finishing this notebook, you should be able to:
- [ ] split your data into training and testing data to test the effectiveness of a certain method
- [ ] apply a simple accuracy function to test the effectiveness of a certain method
- [ ] run multiple classification algorithms:
    - [ ] LASSO
    - [ ] Ridge
    - [ ] ElasticNet
    - [ ] Decision Tree
    - [ ] Random Forest
    - [ ] Nearest Neighbors
    - [ ] Support Vector Machines

# 🥳 One cool finding

We used multiple methods to run classification on the `iris` dataset which is a dataset of flowers and there are three types of iris flowers in it. We split the data into training and testing and ran our methods. Here is the scoreboard:

| method | accuracy score |
|---|---|
| lasso  |1.0|
| ridge  |1.0|
| EN     |1.0|
| DT     |0.960784|
| RF     |0.980392|
| kNN    |1.0|
| SVM    |1.0|